In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
flights = pd.read_csv('nycflights.csv', dtype = {'dep_time' : object, 
                                                 'dep_delay': object, 
                                                 'arr_time' : object, 
                                                 'arr_delay': object, 
                                                 'air_time' : object})
flights['date'] = pd.to_datetime({'year':flights.year, 'month':flights.month, 'day':flights.day})
def convert_time(column_name):
    the_min = flights[column_name][ flights[column_name].notnull() ].astype(str).str[-2:  ]
    the_hr  = flights[column_name][ flights[column_name].notnull() ].astype(str).str[  :-2]
    date_time = pd.to_datetime({'year':flights.year, 'month':flights.month, 'day':flights.day, 
                                'hour': the_hr, 'minute': the_min})
    # the magic of indexes allow us to combine flights.year which has all the rows
    # and the_hr which only exists for rows where dep_time is not NaN
    return date_time
dep_date_time       = convert_time('dep_time')
sched_dep_date_time = convert_time('sched_dep_time')
arr_date_time       = convert_time('arr_time')
sched_arr_date_time = convert_time('sched_arr_time')

datetimes = pd.concat({'dep_date_time': dep_date_time,
           'sched_dep_date_time': sched_dep_date_time, 
           'arr_date_time':arr_date_time, 
           'sched_arr_date_time':sched_arr_date_time}, axis = 1)
flights = flights.join(datetimes)  # we use the .join() function

### A Few Exercises

In [ ]:
flights.info()

In [ ]:
flights.head()

Find all flights that
- Had an arrival delay of two or more hours

In [ ]:
flights.loc[flights.arr_delay.astype(float) > 120].shape

In [ ]:
flights.head()

In [ ]:
(flights.arr_date_time - flights.sched_arr_date_time) > pd.Timedelta(120, 'm')

In [ ]:
# must use timedelta for comparisons between time amounts 
flights.loc[ (flights.arr_date_time - flights.sched_arr_date_time) > pd.Timedelta(120, 'm') ].shape

### There is significant mismatch between these two results.

Let's explore what could be wrong.

In [ ]:
a = flights.loc[flights.arr_delay.astype(float) > 120].index

In [ ]:
b = flights.loc[ (flights.arr_date_time - flights.sched_arr_date_time) > pd.Timedelta(120, 'm') ].index

In [ ]:
a

In [ ]:
a.difference(b)

In [ ]:
flights.loc[151]

the flight departed the next day, and the full date-time column doesn't reflect this


In [ ]:
flights.loc[10455,]

I also found this flight, where the csv file says the departure time is '10'.

It should have read 00:10, as in 12:10AM, but maybe limitations of the csv export dropped the leading 0s.

In [ ]:
a = np.array(['a','','c','d'])
a

In [ ]:
np.where(a == '', '100', a)

In [ ]:
flights = pd.read_csv('nycflights.csv', dtype = {'dep_time' : object,  
                                                 'arr_time' : object, 
                                                 'air_time' : object})
flights['date'] = pd.to_datetime({'year':flights.year, 'month':flights.month, 'day':flights.day})

def convert_time(column_name):
    the_min = flights[column_name][ flights[column_name].notnull() ].astype(str).str[-2:  ]
    the_hr  = flights[column_name][ flights[column_name].notnull() ].astype(str).str[  :-2]
    
    # this following line says that if the hr extracted from strings is empty, '', 
    # then replace it with 00, otherwise, use the hr that appears, and preserve the index
    the_hr  = pd.Series( np.where(the_hr == '', '00',the_hr) , index = the_hr.index)
    
    date_time = pd.to_datetime({'year':flights.year, 'month':flights.month, 'day':flights.day, 
                                'hour': the_hr, 'minute': the_min})
    return date_time

sched_dep_date_time = convert_time('sched_dep_time')
sched_arr_date_time = convert_time('sched_arr_time')

dep_date_time       = sched_dep_date_time + pd.to_timedelta(flights.dep_delay, 'm')
arr_date_time       = sched_arr_date_time + pd.to_timedelta(flights.arr_delay, 'm')

# clean up
# if a flight lands on the following day, we need to update that as well
# this shows that if the arr_date is before the dep_date, add 24 hours to the arrival times
sched_arr_date_time = pd.Series( np.where(sched_arr_date_time < sched_dep_date_time, 
                                          sched_arr_date_time + pd.Timedelta(24, 'h'), 
                                          sched_arr_date_time
                                         ) , index = sched_arr_date_time.index)
arr_date_time = pd.Series( np.where(arr_date_time < dep_date_time,
                                    arr_date_time + pd.Timedelta(24, 'h'),
                                    arr_date_time) , index = arr_date_time.index)

datetimes = pd.concat({'dep_date_time': dep_date_time,
           'sched_dep_date_time': sched_dep_date_time, 
           'arr_date_time':arr_date_time, 
           'sched_arr_date_time':sched_arr_date_time}, axis = 1)
flights = flights.join(datetimes)  # we use the .join() function
flights = flights.drop(['year','month','day'], axis = 1)
airlines = pd.read_csv('airlines.csv', index_col = 'carrier')

In [ ]:
flights.loc[151,]

In [ ]:
flights.loc[10455,]

Note to self: maybe we need to check if it is possible for a flight to be delayed over 24 hours (or maybe they all just get cancelled)

In [ ]:
flights.loc[flights.arr_delay.astype(float) > 120].shape

In [ ]:
flights.loc[ (flights.arr_date_time - flights.sched_arr_date_time) > pd.Timedelta(120, 'm') ].shape

Now our results match!

Find all flights that:
- Flew to Houston (IAH or HOU)
- Were operated by United, American, or Delta
- Departed in summer (July, August, and September)
- Arrived more than two hours late, but didn’t leave late
- Were delayed by at least an hour, but made up over 30 minutes in flight
- Departed between midnight and 6am (inclusive)

In [ ]:
# Flew to Houston (IAH or HOU)
flights.loc[ flights.dest.isin(['IAH','HOU']) ].shape

In [ ]:
# Were operated by United, American, or Delta
flights.loc[ flights.carrier.isin(['UA','AA', 'DL']) ].shape

In [ ]:
# Departed in summer (July, August, and September)
flights.loc[ flights.dep_date_time.dt.month.isin([7,8,9]) ].shape

In [ ]:
# Arrived more than two hours late, but didn’t leave late
# use & for making element-wise boolean comparisons
flights.loc[ (flights.arr_delay > 120) & (flights.dep_delay <= 0) ].head()

In [ ]:
flights.loc[ (flights.arr_delay > 120) & (flights.dep_delay <= 0) ].shape

In [ ]:
# Were delayed by at least an hour, but made up over 30 minutes in flight

flights.loc[ (flights.dep_delay >= 60) & (flights.arr_delay <= flights.dep_delay - 30) ].head()

In [ ]:
flights.loc[ (flights.dep_delay >= 60) & (flights.arr_delay <= flights.dep_delay - 30) ].shape

In [ ]:
# Departed between midnight and 5:59am

flights.loc[  flights.dep_date_time.dt.hour.between(0,5)  ].shape

### Which day of the week is busiest?

In [ ]:
w = flights.groupby(dep_date_time.dt.weekday).date.count()  # monday = 0, sunday = 6
# produces a series
w

In [ ]:
w.index

In [ ]:
d = pd.DataFrame(w)
# rename the index of a dataframe with a dictionary
d.rename({0:'Mon', 1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}, axis = 'index', inplace = True)
d

In [ ]:
d.plot(kind = 'bar', color = 'dodgerblue')
plt.show()

In [ ]:
flights.groupby(dep_date_time.dt.weekday_name).date.count()
# by default it sorts alphabetically

In [ ]:
flights.groupby(dep_date_time.dt.day_name('es')).date.count()  # I later learned of day_name()
# you need to specify the language to use

### Which carriers fly out of which airports?

In [ ]:
# multi-index
flights.groupby(['carrier','origin']).flight.count()

In [ ]:
flights.groupby(['carrier','origin']).flight.count().index

You can make a 'pivot table' by unstacking the multi-index

In [ ]:
flights.groupby(['carrier','origin']).flight.count().unstack()

You can also make a pivot table using the pivot_table method.

In [ ]:
flights.pivot_table('flight',index = 'carrier', columns = 'origin', aggfunc = 'count')

In [ ]:
flights.pivot_table('flight',index = 'carrier', 
                    columns = 'origin', aggfunc = 'count').plot.bar(stacked = True)

In [ ]:
flights.pivot_table('flight',index = 'carrier', 
                    columns = 'origin', aggfunc = 'count').plot.bar()

You can see the top 4 carriers: B6, DL, EV, UA

It appears:
- B6 flies out of JFK the most.
- DL uses JFK and LGA equally
- EV uses EWR heavily
- UA uses EWR heavily as well


I'm curious to see for each of the airports, if one of the carriers will have more 'clout' because it is a bigger user of the airport.

In [ ]:
ewr = flights.loc[flights.origin == 'EWR'].groupby('carrier').flight.count().sort_values(
    ascending = False)
ewr

 I was curious if the proportion of UA and EV flights that are delayed is less than some of the smaller carrier.... maybe these big airlines get higher priority?? 

### Let's look at flights to Los Angeles

In [ ]:
la = flights.loc[flights.dest.isin(['LAX','BUR','ONT','LGB'])]

In [ ]:
la.dest.unique()

In [ ]:
la.dest.value_counts()

In [ ]:
la.carrier.value_counts()

In [ ]:
la.groupby('dest').carrier.value_counts()

B6 (JetBlue) is the only carrier that goes to Burbank or Long Beach. There are no flights from Ontario to New York.

In [ ]:
la.groupby('dest').origin.value_counts()

No direct flights from LaGuardia to Los Angeles. If you want to fly direct to Burbank or Long Beach, you must depart from JFK. 

In [ ]:
la.dep_delay[la.dep_delay.notnull()].astype(int).groupby(la.dest).mean()